In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
YOLO11x で人物だけをトラッキングし，
頭部（バウンディングボックス上部 HEAD_RATIO %）をモザイク処理する。
モザイク付き動画と軌跡（GeoPackage）を出力し，
tqdm のプログレスバーで進捗を確認できるスクリプト。

追加仕様
──────────────────────────────────────────────
* 面積フィルタ：バウンディングボックス面積がフレーム全体の 10 % を超えるものは無視
* Ultralytics の実行ログ (verbose) を抑制し，代わりに tqdm で進捗を表示
"""

import cv2
import pandas as pd
import geopandas as gpd
import numpy as np
from tqdm import tqdm
from ultralytics import YOLO

# ──────────────────────────────────────────────
# 設定パラメータ（必要に応じて変更）
# ──────────────────────────────────────────────
MODEL_PATH   = "yolo11x.pt"          # 学習済みモデル
VIDEO_PATH   = "input1.MP4"
OUTPUT_VIDEO = "output1.mp4"
OUTPUT_GPKG  = "cam_points.gpkg"

HEAD_RATIO      = 0.30      # 上何 % をモザイクにするか
MOS_SCALE       = 0.07      # モザイク粗さ (0.01 細かい, 0.1 粗い)
AREA_FRAC_MAX   = 0.05      # バウンディングボックス面積がフレームの 5 % 超なら無視

BOX_COLOR  = (255, 0, 0)    # BGR
TEXT_COLOR = (255, 255, 255)
THICKNESS  = 2

# ──────────────────────────────────────────────
# モザイク処理関数
# ──────────────────────────────────────────────
def apply_mosaic(img, x1, y1, x2, y2, scale=MOS_SCALE):
    """矩形領域 (x1,y1,x2,y2) をピクセル化して img に書き戻す"""
    h, w = img.shape[:2]
    x1, y1 = max(0, int(x1)), max(0, int(y1))
    x2, y2 = min(w - 1, int(x2)), min(h - 1, int(y2))
    if x2 <= x1 or y2 <= y1:
        return

    roi = img[y1:y2, x1:x2]
    small_w = max(1, int((x2 - x1) * scale))
    small_h = max(1, int((y2 - y1) * scale))
    roi_small  = cv2.resize(roi, (small_w, small_h),
                            interpolation=cv2.INTER_LINEAR)
    roi_mosaic = cv2.resize(roi_small, (x2 - x1, y2 - y1),
                            interpolation=cv2.INTER_NEAREST)
    img[y1:y2, x1:x2] = roi_mosaic

# ──────────────────────────────────────────────
# モデル & 動画 I/O 初期化
# ──────────────────────────────────────────────
model = YOLO(MODEL_PATH)

cap = cv2.VideoCapture(VIDEO_PATH)
frame_width   = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height  = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps           = int(cap.get(cv2.CAP_PROP_FPS))
frame_count   = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # tqdm 用

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out    = cv2.VideoWriter(OUTPUT_VIDEO, fourcc, fps,
                         (frame_width, frame_height))

# ──────────────────────────────────────────────
# メインループ
# ──────────────────────────────────────────────
results_list = []

with tqdm(total=frame_count, desc="Processing") as pbar:
    frame_index = 0
    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break

        # ── 人物 (class 0) のみトラッキング ──
        yolo_results = model.track(
            source=frame,
            persist=True,
            tracker="bytetrack.yaml",
            classes=[0],
            verbose=False            # Ultralytics のログ抑制
        )

        # 検出結果（1フレームにつき 1 Results オブジェクト）
        result = yolo_results[0]

        # ── 各バウンディングボックスの処理 ──
        for box in result.boxes:
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
            conf   = float(box.conf[0])
            obj_id = int(box.id[0]) if box.id is not None else -1
            label  = result.names[int(box.cls[0])]

            # ― 面積フィルタ ―
            area = (x2 - x1) * (y2 - y1)
            if area > AREA_FRAC_MAX * frame_width * frame_height:
                continue   # 5 % 超は無視

            # ― 頭部モザイク ―
            head_y2 = y1 + (y2 - y1) * HEAD_RATIO
            apply_mosaic(frame, x1, y1, x2, head_y2)

            # ― 枠とラベル描画 ―
            cv2.rectangle(frame,
                          (int(x1), int(y1)),
                          (int(x2), int(y2)),
                          BOX_COLOR, THICKNESS)
            text = f"id:{obj_id} {conf:.2f}"
            (tw, th), _ = cv2.getTextSize(text,
                                          cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)
            cv2.rectangle(frame,
                          (int(x1), int(y1 - th - 4)),
                          (int(x1 + tw), int(y1)),
                          BOX_COLOR, -1)
            cv2.putText(frame, text,
                        (int(x1), int(y1 - 2)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6,
                        TEXT_COLOR, 1, cv2.LINE_AA)

            # ― 足元中心点座標（研究用） ―
            x_center = float((x1 + x2) / 2)
            y2_inv   = -float(y2)   # y 軸反転

            results_list.append({
                "Frame": frame_index,
                "ObjectID": obj_id,
                "x": x_center,
                "y": y2_inv
            })

        # 動画へ書き込み
        out.write(frame)

        # Esc で中断可（任意）
        cv2.imshow("YOLO11x Tracking", frame)
        if cv2.waitKey(1) == 27:
            break

        frame_index += 1
        pbar.update(1)

# ──────────────────────────────────────────────
# 後処理
# ──────────────────────────────────────────────
cap.release()
out.release()
cv2.destroyAllWindows()

# DataFrame → GeoDataFrame → GeoPackage
df  = pd.DataFrame(results_list)
gdf = gpd.GeoDataFrame(df,
                       geometry=gpd.points_from_xy(df["x"], df["y"]))
gdf.set_crs("EPSG:6677", allow_override=True, inplace=True)
gdf.to_file(OUTPUT_GPKG, driver="GPKG")
print(f"GeoPackage 形式で結果を {OUTPUT_GPKG} に保存しました。")


Processing: 100%|██████████| 19030/19030 [20:49<00:00, 15.23it/s]


GeoPackage 形式で結果を cam_points.gpkg に保存しました。


In [7]:
import cv2

# 動画ファイルを読み込み
cap = cv2.VideoCapture('output1.mp4')

# 読み込み成功確認
if not cap.isOpened():
    print("動画ファイルを開けませんでした")
else:
    # 最初のフレームを取得（00:00）
    ret, frame = cap.read()

    if ret:
        # フレームのサイズを取得
        height, width = frame.shape[:2]
        print(f"画面サイズ: 幅={width}px, 高さ={height}px")

        # 画像として保存
        cv2.imwrite('back.png', frame)
        print("00:00の画像をback.pngとして保存しました")
    else:
        print("フレームを取得できませんでした")

# 開放
cap.release()


画面サイズ: 幅=2704px, 高さ=1520px
00:00の画像をback.pngとして保存しました
